# Desafio 3 - CDS - Dupla 09

**Integrantes:**

* Lucas Hideki Takeuchi Okamura NUSP: 9274315

* Thales Arantes Kerche Nunes NUSP: 10769372

**Objetivo**

* Programar o algoritmo heurístico CDS, que busca minimizar o makespan em um flow shop com m máquinas.

**Descrição**

* Método heurístico que define uma sequência (idêntica para todas as máquinas), que visa minimizar o tempo total de conclusão das ordens, ou seja, o tempo até a conclusão do último job na última máquina (Cmax);

* O método consiste basicamente em resolver “m-1” subproblemas com n jobs e duas máquinas, utilizando o algoritmo de Johnson e dados derivados problema original.

* O primeiro subproblema consiste em considerar os tempos na primeira máquina e na última máquina;

* O segundo subproblema consiste em considerar a soma dos tempos nas duas primeiras máquinas e a soma dos tempos nas duas últimas;

* Genericamente, o subproblema k, consiste em considerar a soma dos tempos nas k primeiras máquinas e a soma dos tempos nas k últimas máquinas (k=1,2,...,m-1);

* A melhor sequência dentre as “m-1” geradas é tomada como solução do problema original.

In [1]:
import xlwings as xw
import numpy as np

## 1. Definindo Classe _Job_

In [2]:
class Job:
    def __init__(self,i,p):
        self.i=i     # número do job, pela ordem de chegada
        self.p=p     # processing times [p1,p2]
        self.C=[]     # completion times [c1,c2]

## 2. Definindo Funções

**Função para o cálculo dos tempos de conclusão dos jobs em cada uma das m máquinas**

$p_{(j)k}:$ processing time do job da posição j na máquina k

$c_{(j)k}:$ completion time do job da posição j na máquina k

$c_{(1)1} = p_{(1)1}$

$c_{(1)k}=c_{(1),k−1}+p_{(1)k}$, onde    $k=2,...,m$

$c_{(j)1}=c_{(j−1)1}+p_{(j)1}$, onde $j=2,...,n$

$c_{(j)k}=max\left\{{c_{(j),k−1},c_{(j−1)k}}\right\} +p_{(j)k}$, onde $j=2,...,n$ e $k=2,...,m$

In [3]:
def calcCmax(m,jobs):     # esta função calcula os tempos de conclusão dos jobs nas m máquinas
    n=len(jobs)
    c=[[0 for i in range(m)] for j in range(n)]
    p=[job.p for job in jobs]
    
    # primeiro job
    c[0][0]=p[0][0]     # primeira máquina
    for k in range(1,m):     # próximas máquinas
        c[0][k]=c[0][k-1]+p[0][k]
        
    # próximos jobs
    for j in range(1,n):
        c[j][0]=c[j-1][0]+p[j][0]     # primeira máquina
        for k in range(1,m):     # próximas máquinas
            c[j][k]=max(c[j][k-1],c[j-1][k])+p[j][k]
            
    j=0
    for job in jobs:
        job.C=c[j]
        j+=1
    
    return c[-1][-1]

**Algoritmo de Johnson**

In [4]:
def schedJohnson(jobs):
    jobs1=[]
    jobs2=[]
    for job in jobs:
        if job.p[0]<job.p[1]:
            jobs1.append(job)
        else:
            jobs2.append(job)
    
    jobs1.sort(key = lambda job: job.p[0])
    jobs2.sort(key = lambda job: job.p[1], reverse=True)
    
    jobs=jobs1+jobs2
    
    Cmax=calcCmax(2,jobs)     # 2: duas máquinas
    
    return jobs,Cmax

**Algoritmo de Campbell, Dudek e Smith (CDS)**

In [5]:
def schedCDS(jobs, m):
    bestSeq = []
    bestCmax = 9999999999999999999
    for k in range(m - 1):
        a = [Job(job.i, [np.sum(job.p[:k+1]), np.sum(job.p[m-k-1:])]) for job in jobs]     # para cada job, somar os tempos nas k primeiras máquinas
        seq, cmax = schedJohnson(a)
        
        if cmax < bestCmax:
            bestSeq = seq
            bestCmax = cmax
        
        final_jobs = []
        for seq in bestSeq:
            final_jobs.append(jobs[seq.i])
            
        bestCmax = calcCmax(m,final_jobs)
            
    return final_jobs, bestSeq, bestCmax

**Entrada de Dados**

In [6]:
def leInst(i):
    plan = wb1.sheets[i]
    n=int(plan.range('B6').value)
    m=int(plan.range('B5').value)
    tab=plan.range('B11').expand('table').value
    jobs=[]
    for j in range(n):
        p=tab[j]
        jobs.append(Job(j,p))
        
    print()
    print('#','p')
    for job in jobs:
        print(job.i, job.p)
    
    return jobs, m, n

**Saída de resultados**

In [7]:
def gravaSched(i,jobs,Cmax):
    n=len(jobs)
    tab=[]
    for job in jobs:
        a=[job.i+1]+job.p+job.C
        tab.append(a)

    if i==0:
        plan=wb2.sheets[0]
        plan.name='I(1)'
    else:
        plan=wb2.sheets.add('I('+str(i+1)+')',after=i)
        
    plan.range('A5').value=['m',2]
    plan.range('A6').value=['n',n]
    plan.range('E5').value=['Cmax',Cmax]
    plan.range('A10').value=['#','p1','p2','p3','p4','p5','C1','C2','C3','C4','C5']
    plan.range('A11').value=tab

    print()
    print('#','p','C')
    for job in jobs:
        print(job.i, job.p, job.C)
    print('Cmax:',Cmax)

## 3. Testando com dados do Excel

In [8]:
wb1 = xw.Book('xl05 2 A PFSP Cmax CDS.xlsx')
wb2 = xw.Book()
I = wb1.sheets.count     # número de instância é igual ao número de planilhas na pasta

for i in range(I):
    print()
    print(i+1)
    jobs, m, n = leInst(i)
    jobs, bestSeq, Cmax=schedCDS(jobs, m)
    gravaSched(i,jobs,Cmax)
    
wb2.save('xl05 2 A PFSP Cmax CDS.xlsx')


1

# p
0 [58.0, 60.0, 88.0, 14.0, 46.0]
1 [34.0, 33.0, 17.0, 67.0, 15.0]
2 [54.0, 25.0, 93.0, 60.0, 2.0]
3 [19.0, 84.0, 25.0, 47.0, 8.0]
4 [24.0, 45.0, 25.0, 16.0, 11.0]
5 [21.0, 69.0, 93.0, 22.0, 71.0]
6 [64.0, 43.0, 48.0, 83.0, 39.0]
7 [37.0, 43.0, 18.0, 46.0, 13.0]
8 [26.0, 57.0, 34.0, 5.0, 18.0]
9 [82.0, 14.0, 98.0, 27.0, 14.0]
10 [84.0, 43.0, 96.0, 67.0, 81.0]
11 [75.0, 87.0, 45.0, 11.0, 44.0]
12 [73.0, 27.0, 32.0, 9.0, 79.0]
13 [24.0, 94.0, 39.0, 23.0, 95.0]
14 [49.0, 54.0, 8.0, 70.0, 97.0]
15 [27.0, 61.0, 55.0, 87.0, 35.0]
16 [98.0, 8.0, 90.0, 15.0, 55.0]
17 [51.0, 93.0, 80.0, 86.0, 35.0]
18 [83.0, 87.0, 25.0, 94.0, 80.0]
19 [12.0, 80.0, 12.0, 49.0, 38.0]

# p C
19 [12.0, 80.0, 12.0, 49.0, 38.0] [12.0, 92.0, 104.0, 153.0, 191.0]
5 [21.0, 69.0, 93.0, 22.0, 71.0] [33.0, 161.0, 254.0, 276.0, 347.0]
13 [24.0, 94.0, 39.0, 23.0, 95.0] [57.0, 255.0, 294.0, 317.0, 442.0]
15 [27.0, 61.0, 55.0, 87.0, 35.0] [84.0, 316.0, 371.0, 458.0, 493.0]
14 [49.0, 54.0, 8.0, 70.0, 97.0] [133.0, 370.0,


# p
0 [46.0, 28.0, 44.0, 49.0, 38.0]
1 [94.0, 15.0, 6.0, 95.0, 79.0]
2 [98.0, 93.0, 55.0, 47.0, 55.0]
3 [13.0, 15.0, 46.0, 59.0, 52.0]
4 [50.0, 48.0, 7.0, 16.0, 47.0]
5 [82.0, 7.0, 45.0, 53.0, 74.0]
6 [27.0, 91.0, 10.0, 95.0, 11.0]
7 [54.0, 21.0, 60.0, 95.0, 59.0]
8 [86.0, 67.0, 62.0, 85.0, 20.0]
9 [6.0, 24.0, 25.0, 76.0, 65.0]
10 [84.0, 75.0, 82.0, 84.0, 39.0]
11 [66.0, 79.0, 16.0, 80.0, 91.0]
12 [75.0, 64.0, 46.0, 23.0, 19.0]
13 [32.0, 52.0, 67.0, 71.0, 29.0]
14 [30.0, 43.0, 1.0, 5.0, 8.0]
15 [8.0, 21.0, 4.0, 38.0, 35.0]
16 [21.0, 57.0, 80.0, 8.0, 2.0]
17 [54.0, 75.0, 74.0, 61.0, 78.0]
18 [92.0, 83.0, 82.0, 59.0, 83.0]
19 [44.0, 30.0, 61.0, 99.0, 17.0]

# p C
9 [6.0, 24.0, 25.0, 76.0, 65.0] [6.0, 30.0, 55.0, 131.0, 196.0]
15 [8.0, 21.0, 4.0, 38.0, 35.0] [14.0, 51.0, 59.0, 169.0, 231.0]
3 [13.0, 15.0, 46.0, 59.0, 52.0] [27.0, 66.0, 112.0, 228.0, 283.0]
7 [54.0, 21.0, 60.0, 95.0, 59.0] [81.0, 102.0, 172.0, 323.0, 382.0]
17 [54.0, 75.0, 74.0, 61.0, 78.0] [135.0, 210.0, 284.0, 384.0, 46


# p
0 [66.0, 73.0, 83.0, 96.0, 24.0]
1 [69.0, 99.0, 84.0, 96.0, 95.0]
2 [26.0, 4.0, 65.0, 62.0, 59.0]
3 [92.0, 14.0, 5.0, 40.0, 5.0]
4 [55.0, 20.0, 45.0, 74.0, 31.0]
5 [4.0, 24.0, 7.0, 79.0, 41.0]
6 [57.0, 71.0, 59.0, 4.0, 46.0]
7 [52.0, 71.0, 71.0, 88.0, 87.0]
8 [94.0, 55.0, 18.0, 70.0, 25.0]
9 [43.0, 17.0, 12.0, 77.0, 16.0]
10 [44.0, 62.0, 63.0, 80.0, 64.0]
11 [96.0, 49.0, 77.0, 71.0, 89.0]
12 [18.0, 9.0, 17.0, 41.0, 38.0]
13 [91.0, 92.0, 13.0, 74.0, 80.0]
14 [16.0, 84.0, 21.0, 53.0, 88.0]
15 [32.0, 93.0, 65.0, 30.0, 25.0]
16 [45.0, 22.0, 14.0, 94.0, 16.0]
17 [80.0, 5.0, 89.0, 28.0, 69.0]
18 [30.0, 26.0, 73.0, 45.0, 1.0]
19 [71.0, 71.0, 57.0, 35.0, 44.0]

# p C
5 [4.0, 24.0, 7.0, 79.0, 41.0] [4.0, 28.0, 35.0, 114.0, 155.0]
14 [16.0, 84.0, 21.0, 53.0, 88.0] [20.0, 112.0, 133.0, 186.0, 274.0]
12 [18.0, 9.0, 17.0, 41.0, 38.0] [38.0, 121.0, 150.0, 227.0, 312.0]
2 [26.0, 4.0, 65.0, 62.0, 59.0] [64.0, 125.0, 215.0, 289.0, 371.0]
10 [44.0, 62.0, 63.0, 80.0, 64.0] [108.0, 187.0, 278.0, 369.

com_error: (-2147352567, 'Exceção.', (0, 'Microsoft Excel', 'Não é possível salvar esta pasta de trabalho com o mesmo nome de outra pasta de trabalho ou suplemento abertos. Escolha um nome diferente ou feche o documento aberto antes de salvá-lo.', 'xlmain11.chm', 0, -2146827284), None)

In [9]:
ps = [[16, 18, 12],
      [14, 10, 11],
      [13, 20, 15],
      [19, 15, 19],
      [15, 16, 16]]

jobs_test = []
n = len(ps)
m = len(ps[0])

for i in range(n):
    jobs_test.append(Job(i, ps[i]))
    
jobs_test, bestSeq, Cmax = schedCDS(jobs_test, m)

print('#','p','C')
for job in jobs_test:
    print(job.i, job.p, job.C)
print('Cmax:',Cmax)

# p C
2 [13, 20, 15] [13, 33, 48]
4 [15, 16, 16] [28, 49, 65]
3 [19, 15, 19] [47, 64, 84]
0 [16, 18, 12] [63, 82, 96]
1 [14, 10, 11] [77, 92, 107]
Cmax: 107
